# `ner`模型评估

- 数据集
    - 微博`ner`
    - 人民日报
    - `MSRA`
    - `boson`
    
- 模型
    - `random embedding + bilstm-crf`
    - `w2v + bilstm-crf`
    - `bert + bilstm-crf`


In [1]:
from utils.sequence_keras_util import load_data, NerDataGenerator
from keras.layers import Input, Embedding, Bidirectional, LSTM, TimeDistributed, Dense
from keras.models import Model
from keras_contrib.layers import CRF


Using TensorFlow backend.


In [2]:
batch_size = 32
epochs = 20
max_len = 100
embed_dim = 200
lstm_units = 64
proj_units = 64
train_data = "../data/ner/test_data"
val_data = "../data/ner/test_data"
vocab_dic, label_dic = load_data(train_data)
print(label_dic)
print("Vocab Size: %s" % len(vocab_dic))


{'B-ORG': 1, 'I-ORG': 2, 'O': 3, 'B-PER': 4, 'I-PER': 5, 'B-LOC': 6, 'I-LOC': 7}
Vocab Size: 3104


In [3]:
word_input = Input(shape=(max_len,))
model = Embedding(input_dim=len(vocab_dic) + 1, output_dim=embed_dim, input_length=max_len, mask_zero=True)(word_input)
model = Bidirectional(LSTM(units=lstm_units, return_sequences=True, recurrent_dropout=0.1))(model)
model = TimeDistributed(Dense(proj_units, activation="relu"))(model)
crf = CRF(len(label_dic) + 1)
out = crf(model)

model = Model(word_input, out)
model.compile(optimizer="adam", loss=crf.loss_function, metrics=[crf.accuracy])
model.summary()


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 200)          621000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 128)          135680    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 100, 64)           8256      
_________________________________________________________________
crf_1 (CRF)                  (None, 100, 8)            600       
Total params: 765,536
Trainable params: 765,536
Non-trainable params: 0
_______________

D:\programs\conda\lib\site-packages\keras_contrib-2.0.8-py3.7.egg\keras_contrib\layers\crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
D:\programs\conda\lib\site-packages\keras_contrib-2.0.8-py3.7.egg\keras_contrib\layers\crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy


In [4]:
train_fn = NerDataGenerator(train_data, label_dic, vocab_dic, max_len, batch_size)
val_fn = NerDataGenerator(val_data, label_dic, vocab_dic, max_len, batch_size)

hist = model.fit_generator(
    generator=train_fn,
    validation_data=val_fn,
    epochs=epochs)


Instructions for updating:
Use tf.cast instead.
Epoch 1/20
144/144 [==============================] - 185s 1s/step - loss: 8.9318 - crf_viterbi_accuracy: 0.8546 - val_loss: 8.6796 - val_crf_viterbi_accuracy: 0.8980
Epoch 2/20
144/144 [==============================] - 167s 1s/step - loss: 8.6302 - crf_viterbi_accuracy: 0.9085 - val_loss: 8.5277 - val_crf_viterbi_accuracy: 0.9399
Epoch 3/20
144/144 [==============================] - 169s 1s/step - loss: 8.4779 - crf_viterbi_accuracy: 0.9486 - val_loss: 8.4203 - val_crf_viterbi_accuracy: 0.9654
Epoch 4/20
144/144 [==============================] - 168s 1s/step - loss: 8.4494 - crf_viterbi_accuracy: 0.9650 - val_loss: 8.3743 - val_crf_viterbi_accuracy: 0.9744
Epoch 5/20
144/144 [==============================] - 168s 1s/step - loss: 8.4115 - crf_viterbi_accuracy: 0.9728 - val_loss: 8.4050 - val_crf_viterbi_accuracy: 0.9797
Epoch 6/20
144/144 [==============================] - 168s 1s/step - loss: 8.4170 - crf_viterbi_accuracy: 0.9786 - va

In [5]:
x_val, y_val = val_fn.data
print(x_val[0])
print(y_val[0])


[ 1  2  1  3  4  1  5  4  6  7  8  9 10 11 12 13  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0]
[[0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.

In [9]:
import numpy as np

# Eval
pred_val = model.predict(x_val)
pred_val = np.argmax(pred_val, axis=-1)
y_val_true = np.argmax(y_val, axis=-1)

from sklearn_crfsuite.metrics import flat_classification_report

idx2label = {v:k for k, v in label_dic.items()}
idx2label[0] = "PAD"


# Convert the index to tag
pred_tag = [[idx2label[i] for i in row] for row in pred_val]
y_te_true_tag = [[idx2label[i] for i in row] for row in y_val_true] 

report = flat_classification_report(y_pred=pred_tag, y_true=y_te_true_tag)
print(report)


             precision    recall  f1-score   support

      B-LOC       1.00      1.00      1.00      2877
      B-ORG       0.99      0.99      0.99      1331
      B-PER       1.00      1.00      1.00      1973
      I-LOC       1.00      1.00      1.00      4394
      I-ORG       0.99      0.99      0.99      5670
      I-PER       1.00      1.00      1.00      3851
          O       1.00      1.00      1.00    152505
        PAD       1.00      1.00      1.00    290499

avg / total       1.00      1.00      1.00    463100



In [10]:

model.save("../out/ner/crf.h5")
